# Twitter emoji Classifier

In [ ]:
import numpy as np
import pandas as pd
import string

In [ ]:
emoji1=pd.read_csv('/content/Mapping.csv')
emoji1=emoji1.drop(columns='Unnamed: 0')

In [ ]:
emoji1

,emoticons,number
0,😜,0
1,📸,1
2,😍,2
3,😂,3
4,😉,4
5,🎄,5
6,📷,6
7,🔥,7
8,😘,8
9,❤,9


In [ ]:
train=pd.read_csv('/content/Train.csv')


In [ ]:
train=train.drop(columns='Unnamed: 0')

In [ ]:
train

,TEXT,Label
0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,Been friends since 7th grade. Look at us now w...,2
3,This is what it looks like when someone loves ...,3
4,RT @user this white family was invited to a Bl...,3
...,...,...
69995,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3
69996,"I SEA you, Seattle @ Ballard Seafood Festival\n",16
69997,If one of my daughters is wearing this and ask...,2
69998,Guess who whoop people on THEIR homecoming?! #...,3


In [ ]:
train['Label'].value_counts().sort_values(ascending=False)

9     15091
2      7076
3      6842
7      4363
15     3843
13     3250
16     2832
17     2751
1      2592
11     2434
14     2342
6      2083
8      1992
5      1977
12     1909
4      1878
19     1745
18     1722
10     1721
0      1557
Name: Label, dtype: int64

# data imbalance so  we take data Label whoes  count are greater than 3000 and not consider the labels with the highest  rows count


In [ ]:
Labels=[2,3,7,15,13]
filtered_data=train[train['Label'].isin(Labels)]
filtered_emojis=emoji1[emoji1['number'].isin(Labels)]
filtered_emojis=filtered_emojis.reset_index()
filtered_data=filtered_data.reset_index()

In [ ]:
filtered_emojis=filtered_emojis.drop(columns='index')
filtered_data=filtered_data.drop(columns='index')


In [ ]:
filtered_data

,TEXT,Label
0,Been friends since 7th grade. Look at us now w...,2
1,This is what it looks like when someone loves ...,3
2,RT @user this white family was invited to a Bl...,3
3,Maxwell heads home @ Summa Akron City Hospital\n,2
4,"Story On Saturday 136 Collins Ave, Miami Beach...",7
...,...,...
25369,"shout out to all my former clients out there, ...",3
25370,A thing I did today! #Repost @user with @user ...,2
25371,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3
25372,If one of my daughters is wearing this and ask...,2


In [ ]:
filtered_data['Label']=filtered_data['Label'].replace({2:0,3:1,7:2,15:3,13:4})
filtered_emojis['number']=filtered_emojis['number'].replace({2:0,3:1,7:2,13:3,15:4})

In [ ]:
filtered_data['TEXT'][25369]

'shout out to all my former clients out there, and ex coworkers.…\n'

# Data preprocessing

In [ ]:
import re
def remove_tag(data):
    return data.replace('\n',' ')

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=201fe0d65c968015f0f1d27025d08dd4bb95667f732b54521303900aa8b4dcb6
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:

from langdetect import detect

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

In [ ]:
train['TEXT']=train['TEXT'].apply(remove_tag)

In [ ]:
pun=string.punctuation+'…'+'・・・'+'•'+"“"+"”"+"…"
pun

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~…・・・•“”…'

In [ ]:
def remove_punctuation(text):
    translator = str.maketrans("", "", pun)
    return ' '.join(word.translate(translator) for word in text.split())

In [ ]:
filtered_data['preprocessed_data']=filtered_data['TEXT'].apply(remove_punctuation)
filtered_data['preprocessed_data'][25369]

'shout out to all my former clients out there and ex coworkers'

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
ps=PorterStemmer()

In [ ]:
def clean_data(text):
  data=text.lower()
  data=word_tokenize(data)
  y=[]
  for i in data:
    if i.isalnum():
      y.append(i)
  data=y[:]
  y.clear()
  for i in data:
    if i   not in stopwords.words('english'):
      y.append(i)
  return ' '.join(y)



In [ ]:
clean_data('shout out to all my former clients out there, and ex coworkers.…')

'shout former clients ex'

In [ ]:
filtered_data['preprocessed_data']=filtered_data['preprocessed_data'].apply(clean_data)
filtered_data['preprocessed_data']

0        friends since 7th grade look us following drea...
1        looks like someone loves unconditionally oh pu...
2        rt user white family invited black barbecue iv...
3             maxwell heads home summa akron city hospital
4        story saturday 136 collins ave miami beach fl ...
                               ...                        
25369                    shout former clients ex coworkers
25370            thing today repost user user knocking 140
25371    yes call galina bubie go follow beautiful frie...
25372    one daughters wearing asks ice cream 2am wont say
25373    guess whoop people homecoming asurams atlanta ...
Name: preprocessed_data, Length: 25374, dtype: object

# Checking for language of the tweets

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

In [ ]:
filtered_data['Language'] = filtered_data['preprocessed_data'].apply(detect_language)

In [ ]:
filtered_data['Language'].value_counts()

en         20835
af           643
no           490
fr           427
it           384
nl           329
da           308
et           221
ca           218
es           202
cy           190
sv           187
tl           154
ro           129
so           120
id           104
fi            69
pl            51
de            48
pt            46
sl            41
tr            37
hr            30
sw            19
sq            18
lt            16
hu            15
sk            13
lv            13
cs            11
unknown        3
vi             3
Name: Language, dtype: int64

# Filtering out english Tweets

In [ ]:
eng_data=filtered_data[filtered_data['Language']=='en']
eng_data['TEXT'][25369]

'shout out to all my former clients out there, and ex coworkers.…\n'

In [ ]:
eng_data.to_excel('cleaned_data1.xlsx')

In [ ]:
def stem_words(text):
  y=[]
  for words in text:
    y.append(ps.stem(words))
  return ''.join(y)

In [ ]:

eng_data['preprocessed_data']=eng_data['preprocessed_data'].apply(stem_words)
eng_data['preprocessed_data']=eng_data['preprocessed_data'].replace('user','')
eng_data

<ipython-input-45-fff8683c6c21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng_data['preprocessed_data']=eng_data['preprocessed_data'].apply(stem_words)
<ipython-input-45-fff8683c6c21>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng_data['preprocessed_data']=eng_data['preprocessed_data'].replace('user','')


,TEXT,Label,preprocessed_data,Language
0,Been friends since 7th grade. Look at us now w...,0,friends since 7th grade look us following drea...,en
1,This is what it looks like when someone loves ...,1,looks like someone loves unconditionally oh pu...,en
2,RT @user this white family was invited to a Bl...,1,rt user white family invited black barbecue iv...,en
3,Maxwell heads home @ Summa Akron City Hospital\n,0,maxwell heads home summa akron city hospital,en
4,"Story On Saturday 136 Collins Ave, Miami Beach...",2,story saturday 136 collins ave miami beach fl ...,en
...,...,...,...,...
25369,"shout out to all my former clients out there, ...",1,shout former clients ex coworkers,en
25370,A thing I did today! #Repost @user with @user ...,0,thing today repost user user knocking 140,en
25371,"Yes, I call Galina ""my Bubie"" Go follow my bea...",1,yes call galina bubie go follow beautiful frie...,en
25372,If one of my daughters is wearing this and ask...,0,one daughters wearing asks ice cream 2am wont say,en


In [ ]:
cleaned_data=eng_data[['preprocessed_data','Label']]
cleaned_data.to_excel('Cleaned_data.xlsx')

In [ ]:
!pip install emoji

In [ ]:
import emoji

In [ ]:
emoji.demojize(filtered_emojis['emoticons'][0])

':smiling_face_with_heart-eyes:'

In [ ]:
emotions=[]
for i in range(filtered_emojis.shape[0]):
    emotions.append([i,emoji.demojize(filtered_emojis['emoticons'][i])])



In [ ]:
emotions=pd.DataFrame(emotions,columns=['labels','description'])

In [ ]:
emotions=emotions.to_dict()
emotions=emotions['description']

In [ ]:
emotions

{0: ':smiling_face_with_heart-eyes:',
 1: ':face_with_tears_of_joy:',
 2: ':fire:',
 3: ':sparkles:',
 4: ':two_hearts:'}

In [ ]:
emojis1=pd.DataFrame(list(emotions.items()),columns=['number','emoji_meaning'])

In [ ]:
mapped_emoji=filtered_emojis.merge(emojis1,on='number',how='outer')

In [ ]:
mapped_emoji.to_excel('emoji_mapped.xlsx')